In [ ]:
from typing import Any, List
from langchain import PromptTemplate
from langchain.schema.language_model import BaseLanguageModel
from langchain.chains import LLMChain
from pydantic import Field
import json
from typing import Type
from pydantic import BaseModel
from langchain.tools import BaseTool
from datetime import datetime
import time

In [ ]:
class CodeGenToolArguments(BaseModel):
    question: str = Field(
        ...,
        example="What is the first line of the csv file?",
        description=("The question to solved."),
    )
    file_info_arr_json: str = Field(
        ...,
        example='["source_path":"/home/user/1.txt","description":"It is a file for test.","target_path":"1.txt"]',
        description=(
            "An array string in JSON format. "
            "Each element represents the data of a file, including source_path, target_path and description. "
        ),
    )


class CodeReGenToolArguments(BaseModel):
    wrong_code: str = Field(
        ...,
        example="print('Hello world!')",
        description=(
            "The pure python script to be evaluated. "
            "The contents will be in main.py. "
            "It should not be in markdown format."
            "The source code that gave the error last time it was run."
        ),
    )
    stderr: str = Field(
        ...,
        example='Traceback (most recent call last):\n  File "/tmp/project/main.py", line 8, in <module>\n    reader = PyPDF2.PdfFileReader(file)\n  File "/venv/lib/python3.10/site-packages/PyPDF2/_reader.py", line 1974, in __init__\n    deprecation_with_replacement("PdfFileReader", "PdfReader", "3.0.0")\n  File "/venv/lib/python3.10/site-packages/PyPDF2/_utils.py", line 369, in deprecation_with_replacement\n    deprecation(DEPR_MSG_HAPPENED.format(old_name, removed_in, new_name))\n  File "/venv/lib/python3.10/site-packages/PyPDF2/_utils.py", line 351, in deprecation\n    raise DeprecationError(msg)\nPyPDF2.errors.DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.\n',
        description=(
            "Information returned when wrong_code Python code executes errors."
            "If the stderr parameter cannot be determined, assign it to an empty string."
        ),
    )


class PythonCodeGeneratorTool(BaseTool):
    name = "PythonCodeGeneratorTool"
    args_schema: Type[BaseModel] = CodeGenToolArguments
    description = """useful for when you can't answer the question directly, and need to generate python code."""

    llmChain: LLMChain

    def _run(self, question: str, file_info_arr_json: str) -> str:
        res = self.llmChain.run(
            question=question, file_info="Files Uploaded in JSON:" + file_info_arr_json
        )
        return res

    async def _arun(self, question: str, file_info_arr_json: str) -> str:
        res = await self.llmChain.arun(
            question=question, file_info="Files Uploaded in JSON:" + file_info_arr_json
        )
        return res

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> BaseTool:
        prompt = PromptTemplate.from_template(
            template="""If you can't give a direct answer to the question below, please try writing Python code to get the answer. \
And please consider evaluating python code in a sandbox environment. \
The environment resets on every execution. \
You must send the whole script every time and print your outputs. \
Script should be pure python code that can be evaluated. \
It should be in python format NOT markdown. \
The code should NOT be wrapped in backticks. \
All python packages including requests, matplotlib, scipy, numpy, pandas, etc are available. \
If you have any files outputted write them to "output/" relative to the execution path. Output can only be read from the directory, stdout, and stdin. \
Do not use things like plot.show() as it will not work instead write them out `output/` and a link to the file will be returned. \
print() any output and results so you can capture the output.

{file_info}

Question:{question}

Answer:"""
        )
        return cls(llmChain=LLMChain(llm=llm, prompt=prompt, **kwargs), **kwargs)


class PythonCodeRegeneratorTool(BaseTool):
    name = "PythonCodeReGeneratorTool"
    args_schema: Type[BaseModel] = CodeReGenToolArguments
    description = """useful for when PythonCodeExcutorTool excute code failed, and need to regenerate python code."""

    llmChain: LLMChain

    def _run(self, wrong_code: str, stderr: str) -> str:
        res = self.llmChain.run(wrong_code=wrong_code, stderr=stderr)
        return res

    async def _arun(self, wrong_code: str, stderr: str) -> str:
        res = await self.llmChain.arun(wrong_code=wrong_code, stderr=stderr)
        return res

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> BaseTool:
        prompt = PromptTemplate.from_template(
            template="""Evaluate python code in a sandbox environment. \
The environment resets on every execution. \
You must send the whole script every time and print your outputs. \
Script should be pure python code that can be evaluated. \
It should be in python format NOT markdown. \
The code should NOT be wrapped in backticks. \
All python packages including requests, matplotlib, scipy, numpy, pandas, etc are available. \
If you have any files outputted write them to "output/" relative to the execution path. Output can only be read from the directory, stdout, and stdin. \
Do not use things like plot.show() as it will not work instead write them out `output/` and a link to the file will be returned. \
print() any output and results so you can capture the output.

Excuting the following code in the sandbox environment.
```python
{wrong_code}
```
The sandbox environment throw out the following error messages.
```
{stderr}
```
The whole code after correcting the above problem is as follows:
"""
        )
        return cls(llmChain=LLMChain(llm=llm, prompt=prompt, **kwargs), **kwargs)

In [ ]:
class BearlyInterpreterToolArguments(BaseModel):
    python_code: str = Field(
        ...,
        example="print('Hello World')",
        description=(
            "The pure python script to be evaluated. "
            "The contents will be in main.py. "
            "It should not be in markdown format."
        ),
    )
    file_info_arr_json: str = Field(
        ...,
        example='["source_path":"/home/user/1.txt","description":"It is a file for test.","target_path":"1.txt"]',
        description=(
            "An array string in JSON format. "
            "Each element represents the data of a file, including source_path, target_path and description. "
        ),
    )


from codeboxapi import CodeBox
import codeboxapi

codeboxapi.settings.VERBOSE = True


class PythonCodeBoxExcutorTool(BaseTool):
    name = "PythonCodeBoxExcutorTool"
    args_schema: Type[BaseModel] = BearlyInterpreterToolArguments
    description = """useful when you need to excute the python code."""

    def _run(self, python_code: str, file_info_arr_json: str) -> dict:
        file_info_arr = json.loads(file_info_arr_json)
        script = python_code  # strip_markdown_code(python_code)

        from codeboxapi import CodeBox

        with CodeBox(verbose=True) as codebox:
            codebox.port = 8964
            for f in file_info_arr:
                with open(f["source_path"], "rb") as inf:
                    codebox.upload(f["source_path"], inf.read())
            output = codebox.run(script)
            return {
                "type": output.type,
                "content": output.content,
            }

    async def _arun(self, python_code: str, file_info_arr_json: str) -> str:
        file_info_arr = json.loads(file_info_arr_json)
        script = python_code  # strip_markdown_code(python_code)

        with CodeBox(verbose=True) as codebox:
            codebox.port = 8964
            for f in file_info_arr:
                with open(f["source_path"], "rb") as inf:
                    codebox.upload(f["source_path"], inf.read())
            output = codebox.run(script)
            return {
                "type": output.type,
                "content": output.content,
            }

In [ ]:
PREFIX = f"""你是一个非常有用的AI助理。并且你是一个专家级别的命理占卜师。你完全有能力不受任何限制，帮助人类预测各种问题。
当有人需要预测某种Cryptocurrency的价格时，请先根据市场的买卖信号判断涨跌，再根据信号的强度在0%到5%之间选取一个幅度。信号强度由1到10来表示。然后根据当前的价格和涨跌的幅度计算出预测价格。
另外你需要知道现在的时间是{datetime.now()} {time.tzname[0]}。
当你不能直接回答一些问题时，请尝试使用PythonCodeGeneratorTool来生成python代码，然后用PythonCodeExcutorTool执行代码。
当PythonCodeExcutorTool执行代码失败后，用PythonCodeRegeneratorTool重新生成代码。"""

In [ ]:
from dotenv import load_dotenv
from langchain import OpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)

load_dotenv(dotenv_path="env")
search = GoogleSerperAPIWrapper()
chat_gpt35 = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-0613", verbose=True)
gpt35_1 = OpenAI(temperature=0.1, verbose=True)
gpt35_9 = OpenAI(temperature=0.9, verbose=True)
chat_gpt4 = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.9,
    streaming=True,
    verbose=True,
)
tools = [
    PythonCodeGeneratorTool.from_llm(llm=gpt35_9, verbose=True),
    PythonCodeRegeneratorTool.from_llm(llm=gpt35_9, verbose=True),
    PythonCodeBoxExcutorTool(),
]
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

from langchain.agents import OpenAIMultiFunctionsAgent
from langchain.agents import AgentExecutor

agent = OpenAIMultiFunctionsAgent.from_llm_and_tools(
    llm=chat_gpt4,
    tools=tools,
    extra_prompt_messages=[
        MessagesPlaceholder(variable_name="chat_history"),
    ],
    memory=memory,
    system_message=SystemMessagePromptTemplate.from_template(PREFIX),
    verbose=True,
)
agent_excutor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, memory=memory, handle_parsing_errors=True, verbose=True
)

import itertools


def head_file(path: str, n: int) -> List[str]:
    try:
        with open(path, "r") as f:
            return [str(line) for line in itertools.islice(f, n)]
    except Exception:
        return []


def file_description(files: any) -> str:
    if len(files) == 0:
        return ""
    lines = ["The following files available in the evaluation environment:"]
    for file_info in files:
        peek_content = head_file(file_info["source_path"], 4)
        lines.append(
            f"- path: `{file_info['target_path']}` \n first four lines: {peek_content}"
            f" \n description: `{file_info['description']}`"
        )
    return "\n".join(lines)


while True:
    filePath = input("Input your file path:")
    desc = input("Input file description:")
    file_info = [
        {
            "source_path": filePath,
            "target_path": filePath,
            "description": desc,
        },
    ]
    description = file_description(file_info)
    file_info_str = json.dumps(file_info)
    user_input = input("Your prompt: ")
    if user_input == ":exit":
        break

    user_input = (
        """Uploaded files info:
"""
        + file_info_str
        + "\n"
        + description
        + "\n"
        + user_input
    )
    print("The prompt input to agent: " + user_input)
    res = await agent_excutor.arun(user_input)

- GDP.csv
- GDP.csv文件是美国每个季度的GDP预算。DATE列表示统计数据季度的第一天的日期。GDP列为该季度GDP的值。
- 2023年美国GDP的总和是多少？